Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [7]:
from itertools import combinations
from collections import deque
import numpy as np
import random
from geopy.distance import geodesic
import networkx as nx

import pandas as pd
import logging
import pickle
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

from typing import Tuple

logging.basicConfig(level=logging.DEBUG)

# Initialization

In [8]:
def init_all_cities() -> Tuple[dict, dict]:
    ALL_CITIES = {}
    ALL_DIST_MATRIX = {}
    countries = []

    for country in os.listdir('cities'):
        if country.endswith('.csv'):
            country = country[:-4]
            countries.append(country)

    for country in countries:
        # Read the CSV file
        ALL_CITIES[country] = pd.read_csv(f'cities/{country}.csv', header=None, names = ['name', 'lat', 'lon'])

        # Calculate distances between all cities
        cur_dist_matrix = np.zeros((len(ALL_CITIES[country]), len(ALL_CITIES[country])))
        for c1, c2 in combinations(ALL_CITIES[country].itertuples(), 2):
            cur_dist_matrix[c1.Index, c2.Index] = cur_dist_matrix[c2.Index, c1.Index] = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km
        ALL_DIST_MATRIX[country] = cur_dist_matrix

    return ALL_CITIES, ALL_DIST_MATRIX

ALL_CITIES, ALL_DIST_MATRIX = init_all_cities()

In [ ]:
def initialize(country: str):
    CITIES = ALL_CITIES[country]
    DIST_MATRIX = ALL_DIST_MATRIX[country]

    return CITIES, DIST_MATRIX

COUNTRY = 'vanuatu'
CITIES, DIST_MATRIX = initialize(COUNTRY)

logging.info(f'Loaded {len(CITIES)} cities in {COUNTRY}')

INFO:root:Loaded 8 cities in vanuatu


# Helper Functions

### General

In [10]:
def tsp_cost(tsp: list[int]) -> float:
    # Check that the TSP is a cycle and that all cities have been covered
    assert tsp[0] == tsp[-1], f"first city is #{tsp[0]}, last city is #{tsp[-1]}"
    # logging.debug(f"set(tsp): {set(tsp)}\nset(range(len(tsp))): {set(range(len(tsp)))}")
    assert set(tsp) == set(range(len(tsp) - 1)), f"tsp covers {len(tsp)}, should cover {len(tsp) - 1}"

    # Finally, compute the total cost
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]

    return tot_cost

def print_tsp(tsp: list):
    first = True
    for city in tsp:
        if first:
            prev_city = city
            first = False
            continue

        logging.info(f"step: {CITIES.at[prev_city,'name']} -> {CITIES.at[city,'name']} ({DIST_MATRIX[prev_city, city]:.2f}km)")
        prev_city = city

    logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")



### Segments

In [11]:
def tsp_segment_to_series(tsp: list[tuple]) -> list[int]:
    # Convert the list of segments to a list of cities
    tsp_series = [t[0] for t in tsp]
    tsp_series.append(tsp[0][0])
    return tsp_series


# def print_tsp_segments(tsp: list):
#     # logging.info(f"Total cost: {tsp_cost(tsp):.2f}km")

#     for segment in tsp:
#         city1, city2 = segment
#         logging.info(f"step: {CITIES.at[city1,'name']} -> {CITIES.at[city2,'name']} ({DIST_MATRIX[city1, city2]:.2f}km)")


def complete_segment_tsp(tsp: list[tuple], degrees: list[int]) -> list[tuple]:
    assert len(degrees) == len(CITIES), f"len(degrees)={len(degrees)} != len(CITIES)={len(CITIES)}"
    assert all (degree > 0 for degree in degrees), f"degrees={degrees} should be all > 0"

    indices_with_value_1 = [index for index, value in enumerate(degrees) if value == 1]
    assert len(indices_with_value_1) == 2, f"len(indices_with_value_1)={len(indices_with_value_1)} != 2"
    tsp.append((indices_with_value_1[0], indices_with_value_1[1]))

    return tsp

def order_segments(segments: list[tuple]) -> list[tuple]:
    # Create a dictionary to map each node to its connected nodes
    connections = {}
    for u, v in segments:
        if u in connections:
            connections[u].append(v)
        else:
            connections[u] = [v]
        if v in connections:
            connections[v].append(u)
        else:
            connections[v] = [u]

    # Start with the first segment in the list
    ordered_segments = [segments[0]]
    current = segments[0][1]

    # Reorder the list by following the connections
    while len(ordered_segments) < len(segments):
        next_city = connections[current].pop()
        connections[next_city].remove(current)
        next_segment = (current, next_city)
        ordered_segments.append(next_segment)
        current = next_city

    return ordered_segments


# Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

### Greedy Solution 1 - Closest Cities Trasversal
The first greedy solution is able to find performance that is almost on par with that of the second one (Shortest Segment) by using an incredibly small fraction of the second's computing time (e.g. 1.2s vs 10m for the China instance)

In [12]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True

tsp = list()
tsp.append(int(city))

while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)")
    
    visited[closest] = True
    city = closest
    tsp.append(int(city))
    
logging.debug(f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)")
tsp.append(tsp[0])

tot_cost = tsp_cost(tsp)
logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")

with open(f'{COUNTRY}_tsp_greedy1.pkl', 'wb') as file:
    pickle.dump(tsp, file)
tsp_greedy_1 = tsp

DEBUG:root:step: Isangel -> Vila (223.00km)
DEBUG:root:step: Vila -> Lakatoro (206.74km)
DEBUG:root:step: Lakatoro -> Norsup (2.46km)
DEBUG:root:step: Norsup -> Luganville (67.09km)
DEBUG:root:step: Luganville -> Port Olry (52.02km)
DEBUG:root:step: Port Olry -> Longana (105.77km)
DEBUG:root:step: Longana -> Sola (165.49km)
DEBUG:root:step: Sola -> Isangel (652.96km)
INFO:root:result: Found a path of 8 steps, total length 1475.53km


### Greedy Solution 2: Shortest Segments

In [ ]:
# Define the segments as ((city1, city2), distance) nested tuples
all_segments = list(combinations(range(len(CITIES)), 2))
all_segments = [((i, j), DIST_MATRIX[i, j]) for i, j in all_segments]
# Then define the tsp as a list of segments and the union-find data structure
uf = nx.utils.UnionFind()
tsp = list()
cities_degrees = [0] * len(CITIES) # Degree as in how many connections/segments

all_segments_empty = False
while not all_segments_empty:
    # Extract (i.e. also remove) the shortest segment
    shortest_seg = min(all_segments, key=lambda x: x[1])
    all_segments.remove(shortest_seg)
    shortest_seg = shortest_seg[0]  # Remove the distance from the tuple

    # Check if the two cities are already connected AND if the vertices are peripheral
    different_sets = uf[shortest_seg[0]] != uf[shortest_seg[1]]
    peripheral_vertices = cities_degrees[shortest_seg[0]] < 2 and cities_degrees[shortest_seg[1]] < 2
    if different_sets and peripheral_vertices:
        # Update the uf and the tsp
        uf.union(*shortest_seg)         # * is the unpacking operator
        tsp.append(shortest_seg)
        # Update the degrees of the cities
        cities_degrees[shortest_seg[0]] += 1
        cities_degrees[shortest_seg[1]] += 1

    all_segments_empty = len(all_segments) == 0
    
# Close the cycle and reorder it 
tsp = complete_segment_tsp(tsp, cities_degrees)
tsp = order_segments(tsp)

# Convert the list of segments to a list of cities
tsp = tsp_segment_to_series(tsp)
print_tsp(tsp)

with open(f'./pickles/{COUNTRY}_tsp_greedy2.pkl', 'wb') as file:
    pickle.dump(tsp, file)

tsp_greedy_2 = tsp


INFO:root:step: Lakatoro -> Norsup (2.46km)
INFO:root:step: Norsup -> Luganville (67.09km)
INFO:root:step: Luganville -> Port Olry (52.02km)
INFO:root:step: Port Olry -> Longana (105.77km)
INFO:root:step: Longana -> Sola (165.49km)
INFO:root:step: Sola -> Isangel (652.96km)
INFO:root:step: Isangel -> Vila (223.00km)
INFO:root:step: Vila -> Lakatoro (206.74km)
INFO:root:result: Found a path of 8 steps, total length 1475.53km


### Comparison

In [14]:
def load_all_pkl_files(directory):
    # Take only files containing .pkl and greedy
    pkl_files = [f for f in os.listdir(directory) if f.endswith('.pkl')]
    pkl_files = [f for f in pkl_files if 'greedy' in f]

    # Load all the data
    data = {}
    for pkl_file in pkl_files:
        with open(os.path.join(directory, pkl_file), 'rb') as file:
            data[pkl_file] = pickle.load(file)
    return data


directory = '.'
all_data = load_all_pkl_files(directory)

times = {"china_tsp_greedy1.pkl": 1.2,
         "china_tsp_greedy2.pkl": 6142.5,
         "italy_tsp_greedy1.pkl": 0.1,
         "italy_tsp_greedy2.pkl": 0.1,
         "russia_tsp_greedy1.pkl": 0.2,
         "russia_tsp_greedy2.pkl": 15.3,
         "us_tsp_greedy1.pkl": 0.5,
         "us_tsp_greedy2.pkl": 180.0,
         "vanuatu_tsp_greedy1.pkl": 0.0,
         "vanuatu_tsp_greedy2.pkl": 0.0}

plt.figure(figsize=(12, 8))

# Define the legend patches
red_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='r', markersize=10, label='Greedy1')
blue_patch = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='b', markersize=10, label='Greedy2')

# Plot the scatter points
for name, tsp in tqdm(all_data.items(), desc="Processing TSP files"):
    CITIES, DIST_MATRIX = initialize(name.split('_')[0])

    cost = tsp_cost(tsp)
    time = times[name]
    color = 'r' if 'greedy1' in name else 'b'

    plt.scatter(cost, time, color=color)
    plt.text(cost, time, name.split('_')[0], fontsize=9)

# Add the legend patches
plt.legend(handles=[red_patch, blue_patch])
plt.ylabel('Time (s)')
plt.xlabel('Cost (km)')
plt.title('TSP Costs vs. Time for Greedy Solutions')
plt.show()

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


Processing TSP files:   0%|          | 0/10 [00:00<?, ?it/s]DEBUG:matplotlib.font_manager:findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\Dragos\\Roba\\Lectures\\YM2.1\\Computational Intelligence\\b Labs\\CI2024_lab2\\.venv\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\cmb10.ttf', name='cmb10', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\Dragos\\Roba\\Lectures\\YM2.1\\Computational Intelligence\\b Labs\\CI2024_lab2\\.venv\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\cmex10.ttf', name='cmex10', style='normal', variant='normal', weight=400, stretch='normal', size='scalable')) = 10.05
DEBUG:matplotlib.font_manager:findfont: score(FontEntry(fname='c:\\Users\\Dragos\\Roba\\Lectures\\YM2.1\\Computational Intelligence\\b Labs\\CI2024

As one can see, the extra time needed to compute the slightly better solution using greedy solution 2 is not worth it probably.  
Keep in mind though that the implementation of the second greedy solution could be optimized, maybe even by a lot, but I believe that the results would still hold, based on the level of efficiency I estimated my developed code to have.

This could be already considered an example analysis of the goal of the current laboratory, which was stated to be — alongside the exploration of solutions to the tsp — the finding of solutions that have slightly worse results than the best presented, but can find them much faster. Anyway, I will keep exploring other solutions using evolution and genetic operators.